In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder('data', transform=transform)

In [4]:
TRAIN_PORTION = 0.8

train_size = int(TRAIN_PORTION * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [5]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [6]:
class FaceClasifier(nn.Module):
	def __init__(self):
		super(FaceClasifier, self).__init__()
		self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1)
		self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=2)
		self.pool = nn.MaxPool2d(3, 3)
		self.fc1 = nn.Linear(64 * 32 * 32, 128)
		self.fc2 = nn.Linear(128, 2)
		self.dropout = nn.Dropout(0.25)
		self.relu = nn.LeakyReLU()

	def forward(self, x):
		x = self.pool(self.relu(self.conv1(x)))
		x = self.pool(self.relu(self.conv2(x)))
		x = x.view(-1, 64 * 32 * 32)
		x = self.relu(self.fc1(x))
		x = self.fc2(x)
		return x

In [7]:
model = FaceClasifier().to(device)

In [8]:
for name, param in model.named_parameters():
	# print parameter name and size
	print(name, param.numel())
print('------------------')
print(f'Total parameters: {sum(p.numel() for p in model.parameters()):,}')

conv1.weight 864
conv1.bias 32
conv2.weight 51200
conv2.bias 64
fc1.weight 8388608
fc1.bias 128
fc2.weight 256
fc2.bias 2
------------------
Total parameters: 8,441,154


In [9]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
num_epochs = 10
from collections import defaultdict

train_losses = defaultdict(list)

for epoch in range(num_epochs):
	num_batches = len(train_loader)
	for i, (inputs, label) in tqdm(enumerate(train_loader, 1), total=num_batches, desc=f"Epoch {epoch + 1}/{num_epochs}"):
		inputs = inputs.to(device)
		label = label.to(device)
		optimizer.zero_grad()
		outputs = model(inputs)
		loss = criteria(outputs, label)
		loss.backward()
		optimizer.step()
		train_losses["batch"].append(i + num_batches * epoch)
		train_losses["loss"].append(loss.item())

print('Finished Training')

Epoch 1/10:   0%|          | 0/26 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
# test accuracy
correct = 0
total = 0
for inputs, label in test_loader:
	inputs = inputs.to(device)
	label = label.to(device)
	outputs = model(inputs)
	_, predicted = torch.max(outputs.data, 1)
	total += label.size(0)
	correct += (predicted == label).sum().item()

print(f'Accuracy of the network on the {total} test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 409 test images: 57.95%


In [23]:
# train accuracy
correct = 0
total = 0
for inputs, label in train_loader:
	inputs = inputs.to(device)
	label = label.to(device)
	outputs = model(inputs)
	_, predicted = torch.max(outputs.data, 1)
	total += label.size(0)
	correct += (predicted == label).sum().item()

print(f'Accuracy of the network on the {total} train images: {100 * correct / total:.2f}%

Accuracy of the network on the 1632 train images: 83.33%
